# Исследование опроса клиентов телекомунникацонной компании

# Оглавление   
[1 Описание проекта](#description)   
[2 Описание данных](#data)    
[3 Выгрузка данных](#preparing)      
[4  Создание дашборда в Tableau](#tableau)     
[5  Общий вывод](#result)    

### Описание проекта <a id="description"></a>
Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым». Компания провела опрос и попросила меня подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

В ходе исследования ответим на следующие вопросы:

- Как распределены участники опроса по возрасту и полу?    
- Каких пользователей больше: новых или старых?    
- Пользователи из каких городов активнее участвовали в опросе?    
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?    
- Какой общий NPS среди всех опрошенных?    
- Как можно описать клиентов, которые относятся к группе cторонников?    


Исследование проведем в два этапа:

- Выгрузка данных    
- Создание дашборда в Tableau    

### Описание данных <a id="data"></a>

Таблица **user**    
Содержит основную информацию о клиентах.
- user_id -	Идентификатор клиента, первичный ключ таблицы
- lt_day -	Количество дней «жизни» клиента
- age -	Возраст клиента в годах
- gender_segment -	Пол клиента (1 – женщина, 0 – мужчина)
- os_name -	Тип операционной системы
- cpe_type_name -	Тип устройства
- location_id -	Идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
- age_gr_id -	Идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
- tr_gr_id -	Идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице traffic_segment
- lt_gr_id -	Идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment
- nps_score -	Оценка клиента в NPS-опросе (от 1 до 10)


Таблица **location**    
Справочник территорий, в которых телеком-компания оказывает услуги.
- location_id -	Идентификатор записи, первичный ключ
- country -	Страна
- city -	Город


Таблица **age_segment**    
Данные о возрастных сегментах клиентов.
- age_gr_id -	Идентификатор сегмента, первичный ключ
- bucket_min -	Минимальная граница сегмента
- bucket_max -	Максимальная граница сегмента
- title -	Название сегмента


Таблица **traffic_segment**    
Данные о выделяемых сегментах по объёму потребляемого трафика.
- tr_gr_id -	Идентификатор сегмента, первичный ключ
- bucket_min -	Минимальная граница сегмента
- bucket_max -	Максимальная граница сегмента
- title -	Название сегмента


Таблица **lifetime_segment**    
Данные о выделяемых сегментах по количеству месяцев «жизни» клиента — лайфтайму.
- lt_gr_id -	Идентификатор сегмента, первичный ключ
- bucket_min -	Минимальная граница сегмента
- bucket_max -	Максимальная граница сегмента
- title -	Название сегмента

### Выгрузка данных <a id="preparing"></a>

Для проведения исследования выгрузим следующие поля:    
- user_id - Идентификатор клиента
- lt_day - Количество дней «жизни» клиента
- is_new -	Поле хранит информацию о том, является ли клиент новым
- age -	Возраст
- gender_segment -	Пол
- os_name -	Тип операционной системы
- cpe_type_name -	Тип устройства
- country -	Страна проживания
- city -	Город проживания
- age_segment -	Возрастной сегмент
- traffic_segment -	Сегмент по объёму потребляемого трафика
- lifetime_segment -	Сегмент по количеству дней «жизни»
- nps_score -	Оценка клиента в NPS-опросе
- nps_group -	Поле хранит информацию о том, к какой группе относится оценка клиента в опросе

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'TRUE'
           ELSE  'FALSE'
       END AS is_new,
       u.age,
       CASE
           WHEN u.gender_segment == 1 THEN '1'
           WHEN u.gender_segment == 0 THEN '0'
           ELSE 'NaN'
       END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       substr(a.title, 4) AS age_segment,
       substr(t.title, 4) AS traffic_segment,
       substr(life.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE
           WHEN u.nps_score >= 9 AND u.nps_score <= 10 THEN 'сторонники'
           WHEN u.nps_score >= 7 AND u.nps_score <= 8 THEN 'нейтралы'
           ELSE  'критики'
       END AS nps_group
FROM user AS u LEFT JOIN location AS l ON u.location_id = l.location_id
               LEFT JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
               LEFT JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
               LEFT JOIN lifetime_segment AS life ON u.lt_gr_id = life.lt_gr_id
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,FALSE,45.0,1,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,FALSE,53.0,0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,FALSE,57.0,0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

**Вывод:**    
Загрузка данных прошла без ошибок, сформированы необходимые для исследования категории, итоговая таблица сохранена в файл `csv`  и подготовлена для работы в Tableau

### Создание дашборда в Tableau <a id="tableau"></a>

Дашборд "Исследование потребительской лояльности":<p>
    
https://public.tableau.com/views/telecomm_NPS/sheet31?:language=en-US&:display_count=n&:origin=viz_share_link    

### Общий вывод <a id="result"></a>

В опросе приняли участие 502493 пользователя, основная их часть (80%) имеет возраст от 25 до 55 лет. Женщин участвовало в опросе на 10% больше чем мужчин. Более 82% опрошенных являются старыми пользователями, а 54% опрошенных пользуются услугами компании дольше 36 месяцев. Активнее всего в опросе участвовали жители крупных городов, среди которых Москва ожидаемо заняла первое место.

Женщины более лояльны к сервису чем мужчины, а новыве пользователи значительно лояльнее пользующихся сервисом более 36 месяцев. У клиентов младше 16 лет процент критиков превышает процент сторонников, но с возрастом лояльность пользователей увеличивается. В Иваново самые лояльные пользователи, большинство городов с наиболее лояльными клиентами находятся в европейской части страны.

Общий NPS среди опрошенных равен 21.9, более половины из них являются сторонниками компании. При этом лояльность снижена у старых клиентов и у пользователей IOS. Клинты, не потребляющие трафик, или использующие слишком много трафика, также вносят вклад в снижение общей лояльности.

Среди опрошенных, сторонником компании является 264081 клиент - чаще это женщины (56%), пользующиеся операционной системой Android (88%), их возраст от 25 до 55 лет (80%). Половина сторонников (51%) пользуются услугами компании более 36 месяцев и используют объём трафика ниже среднего, с ростом трафика количество сторонников уменьшается.